# Finding high-frequency words tweets
In this notebook, a map-reduce procedure is used here to find words which have a high frequency in the unique tweets (this means retweets are excluded from analysis).

Download a tweets from  json from archive.org (https://archive.org/search.php?query=tweets). For every given minute there is .bz2 file and all files for every hour is given in separate folders. These files contains tweets in form of json.

Mapper and reducer files are below. The analysis is for one hour of tweets.
The first line in the cell save the rest of the cell as .py file. If you want to do this manually, remove the first line and save the cell content.

In [150]:
%%writefile mapper_vocab_freq.py 
#!/usr/bin/env python
import sys
import json

      
for currentTweet in sys.stdin:
    #try:
    if currentTweet.strip() != "":
        currentTweet = currentTweet.lower().strip()  
        json_dic = json.loads(currentTweet)
        if 'text' in json_dic:
            currentTweetText = json_dic['text']
            # The following line replace all special character in the string with a space
            currentTweetText = currentTweetText.translate ({ord(ch): " " for ch in '"\'!@#$%^&*()[]{};:,./<>?\|`~-=_+0123456789'})
            list_of_words = currentTweetText.lower().split()
            for word in list_of_words:
                print(word,1)
    #except:
    #    print('Error',1)

Overwriting mapper_vocab_freq.py


In [145]:
%%writefile reducer.py 
#!/usr/bin/env python
# source for reducer file: https://www.michael-noll.com/tutorials/writing-an-hadoop-mapreduce-program-in-python/
import sys

current_word = None
current_count = 0
word = None

for line in sys.stdin:
    line = line.strip()
    word, count = line.split(' ', 1)
    try:
        count = int(count)
    except ValueError:
        print('count_error')
    if current_word == word:
        current_count += count
    else:
        if current_word:
            text_to_print = current_word + ',' + str(current_count)
            print(text_to_print)
            #print(current_word, current_count)
        current_count = count
        current_word = word

if current_word == word:
    text_to_print = current_word + ',' + str(current_count)
    print(text_to_print)

Overwriting reducer.py


### run map reduce 
 run the following command in the command line:
 
 *bzcat *.bz2 | ./mapper_unique_tweets.py | sort -k1,1 | ./reducer.py*
 
 Another alternative is to write the above command in a shell script as follows. This .sh file write the result in output.txt

In [146]:
%%writefile mapreduce.sh
echo 'word,freq' > output.csv
bzcat *.bz2 | ./mapper_vocab_freq.py | sort -k1,1 | ./reducer.py >> output.csv

Overwriting mapreduce.sh


After that you can run the .sh file and print the output text.

In [147]:
!mapreduce.sh
import csv
d = {}
with open('output.csv', newline='\n') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        d[row['word']] = int(row['freq'])
dict(sorted(d.items(), key=lambda item: item[1],reverse=True))

{'Error': 14268,
 'rt': 9446,
 't': 4058,
 'https': 3659,
 'co': 3636,
 'the': 2105,
 'a': 1778,
 'i': 1708,
 'to': 1650,
 'you': 1283,
 'and': 1172,
 'is': 1108,
 'of': 1050,
 'for': 956,
 'in': 935,
 'this': 683,
 'de': 639,
 'it': 635,
 's': 632,
 'my': 602,
 'me': 585,
 'on': 569,
 'that': 531,
 'be': 529,
 'got': 514,
 'with': 464,
 'your': 449,
 'are': 433,
 'm': 407,
 'no': 398,
 'so': 359,
 'que': 358,
 'la': 346,
 'more': 342,
 'not': 338,
 'we': 336,
 'y': 315,
 'u': 313,
 'official': 309,
 'love': 303,
 'can': 299,
 'if': 298,
 'like': 291,
 'd': 283,
 'will': 278,
 'as': 275,
 'have': 274,
 'but': 272,
 'what': 271,
 'at': 265,
 'from': 262,
 'all': 259,
 'e': 258,
 'en': 257,
 'do': 256,
 'txt': 256,
 'day': 248,
 'was': 248,
 'enhypen': 247,
 'new': 247,
 'just': 242,
 'he': 240,
 'up': 235,
 'they': 229,
 'amp': 226,
 'one': 224,
 'now': 218,
 'o': 216,
 'v': 213,
 'us': 212,
 'k': 211,
 'el': 203,
 '…': 203,
 'z': 196,
 'by': 193,
 'n': 193,
 'about': 190,
 'who': 189,
